In [1]:
import pandas as pd
df = pd.read_excel("CeaData.xlsx")
df

,Role,Experience,Performance,Internet,Location,Approval
0,Technical,Senior,Excellent,Good,Urban,Yes
1,Technical,Junior,Excellent,Good,Urban,Yes
2,Non-Technical,Junior,Average,Poor,Rural,No
3,Technical,Senior,Average,Good,Rural,No
4,Technical,Senior,Excellent,Good,Rural,Yes


In [2]:
X = df.drop("Approval",axis=1)
y = df["Approval"]
columns = df.columns.tolist()
print(columns)

['Role', 'Experience', 'Performance', 'Internet', 'Location', 'Approval']


In [3]:
num_attributes = len(columns) - 1
S = ["Ø"] * num_attributes
G = [["?"] * num_attributes]
print("S0:", S)
print("G0:", G)

S0: ['Ø', 'Ø', 'Ø', 'Ø', 'Ø']
G0: [['?', '?', '?', '?', '?']]


In [4]:
def is_consistent(hypothesis, instance):
    for h, x in zip(hypothesis, instance):
        if h != "?" and h != x:
            return False
    return True

In [5]:
def is_more_general(h1, h2):
    for a, b in zip(h1, h2):
        if a != "?" and a != b:
            return False
    return True

In [9]:
for index, row in df.iterrows():
    instance = row[:-1].tolist()
    label = row[-1]
    print(f"\nExample {index+1}: {instance} → {label}")

    if label == "Yes":
        for i in range(num_attributes):
            if S[i] == "Ø":
                S[i] = instance[i]
            elif S[i] != instance[i]:
                S[i] = "?"
        G = [g for g in G if is_consistent(g, instance)]
    else:
        G_new = []
        for g in G:
            if not is_consistent(g, instance):
                G_new.append(g)
            else:
                for i in range(num_attributes):
                    if g[i] == "?" and S[i] != "?" and S[i] != instance[i]:
                        new_h = g.copy()
                        new_h[i] = S[i]
                        if is_more_general(new_h, S):
                            G_new.append(new_h)
        G_unique = []
        for h in G_new:
            if h not in G_unique:
                G_unique.append(h)
                
        G_pruned = []
        for h in G_unique:
            if not any(is_more_general(other, h) and other != h for other in G_unique):
                G_pruned.append(h)
        G = G_pruned
    print("S:", S)
    print("G:", G)


Example 1: ['Technical', 'Senior', 'Excellent', 'Good', 'Urban'] → Yes
S: ['Technical', '?', 'Excellent', 'Good', '?']
G: [['Technical', '?', 'Excellent', '?', '?'], ['?', '?', 'Excellent', '?', '?'], ['?', '?', 'Excellent', 'Good', '?']]

Example 2: ['Technical', 'Junior', 'Excellent', 'Good', 'Urban'] → Yes
S: ['Technical', '?', 'Excellent', 'Good', '?']
G: [['Technical', '?', 'Excellent', '?', '?'], ['?', '?', 'Excellent', '?', '?'], ['?', '?', 'Excellent', 'Good', '?']]

Example 3: ['Non-Technical', 'Junior', 'Average', 'Poor', 'Rural'] → No
S: ['Technical', '?', 'Excellent', 'Good', '?']
G: [['?', '?', 'Excellent', '?', '?']]

Example 4: ['Technical', 'Senior', 'Average', 'Good', 'Rural'] → No
S: ['Technical', '?', 'Excellent', 'Good', '?']
G: [['?', '?', 'Excellent', '?', '?']]

Example 5: ['Technical', 'Senior', 'Excellent', 'Good', 'Rural'] → Yes
S: ['Technical', '?', 'Excellent', 'Good', '?']
G: [['?', '?', 'Excellent', '?', '?']]


C:\Users\Home\AppData\Local\Temp\ipykernel_12632\2493156365.py:3: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  label = row[-1]


In [10]:
print("\nFinal Specific Boundary (S):", S)
print("Final General Boundary (G):", G)


Final Specific Boundary (S): ['Technical', '?', 'Excellent', 'Good', '?']
Final General Boundary (G): [['?', '?', 'Excellent', '?', '?']]


In [11]:
def predict(instance, S, G):
    s_match = is_consistent(S, instance)
    g_match = all(is_consistent(g, instance) for g in G)

    if s_match and g_match:
        return "Yes"
    elif not g_match:
        return "No"
    else:
        return "Uncertain"
new_instance = ["Technical", "Senior", "Excellent", "Good", "Urban"]
prediction = predict(new_instance, S, G)
print("Prediction for new instance:", prediction)


Prediction for new instance: Yes
